In [1]:
installPackage "NumericalAlgebraicGeometry";

stdio:2:1:(3): error: installPackage: the package is already installed and loaded from /Users/timcheek/Library/Application Support/Macaulay2/local/share/Macaulay2/


In [4]:
R2=CC[x];
R1=CC[x][t];
use R1;

In [5]:
taylorPoly=(f, x0, t0) -> {
    Fx=diff(x,f);   --yes, standard form is diff(x,f) for differentiate f wrt x
    Ft=diff(t,f);
    --yes, standard form is sub(f,{t=>2}), which works
    --also, need to do division in CC, otherwise get weird fraction field stuff
    g=x0-sub(sub(Ft,{x=>x0,t=>t0}),CC)/sub(sub(Fx,{x=>x0,t=>t0}),CC)*(t-t0);   
    return g;
};

In [6]:
runCorrector=(F, newT, guess)-> {
    f=sub(sub(F,{t=>newT}),R2);    
    guess=point{{guess}};
    for i from 1 to 99 do (
        guess=newton(polySystem{f},guess);
        
        --print(guess.Coordinates);
    );
    return guess;
};

In [134]:
runAlgo=(F,x0,t0, newP1, newP2)->{
    approxFunc=taylorPoly(F,x0,t0);
    
    --now, substitute in other portal t1 value into g(t), to get approx zero for F(x,t1)=0 system
    --note that the complex number is represented by ii

    zeroGuess=sub(approxFunc,{t=>newP1});
    
    --now, apply newtons to the F(x,newP1)=0 system, using the zeroGuess estimate
    
    newX1= runCorrector(F, newP1, zeroGuess);
    --newX1.Coordinates_0;
    --now, we have that (newX, newP) is a different solution to F(x,t)=0 (at least it should be)
    
    approxFunc=taylorPoly(F,newX1.Coordinates_0, newP1);
    zeroGuess=sub(approxFunc, {t=>newP2});
    newX2=runCorrector(F, newP2, zeroGuess);
    
    --zeroGuess=sub(approxFunc, {t=>t0});
    --newX2=runCorrector(F, t0, zeroGuess);
    --return newX2;
    
    approxFunc=taylorPoly(F,newX2.Coordinates_0, newP2);
    zeroGuess=sub(approxFunc, {t=>t0});
    return runCorrector(F, t0, zeroGuess);
    
};

In [139]:
F=(x^3-3*x-t)_R1;
x0=0;
t0=0;

guessPoint=runAlgo(F,x0,t0,2+0.5*ii,2-0.5*ii);
guessPoint.Coordinates_0






o139 = -1.73205

o139 : CC  [x]
         53


In [62]:
--THIS IS A 2-GON
F=(x^3-3*x-t)_R1;
x0=0;
t0=0;

s=10;
V=mutableIdentity(R1,2*s+1);
Coords=mutableIdentity(R1,2*s+1);
print V;
for j from 0 to 2*s do (
    for i from 0 to 2*s do (
        try(
            newSol=runAlgo(F,x0,t0,(-5+0.5*i)+(5-0.5*j)*ii,t0);
        
            
        
            
        ) then (
            print(newSol.Coordinates_0);
            V_(i,j)= newSol.Coordinates_0;
        ) else (
            V_(i,j)= 9;
        );
        Coords_(i,j)=(-5+0.5*i)+(5-0.5*j)*ii;
    );
);
print V;
print Coords;






stdio:143:20:(3): error: no method found for applying promote to:
     argument 1 :  0 (of class CC  [x])
                                 53
     argument 2 :  CC
                     53

| 1 0 0 |
| 0 1 0 |
| 0 0 1 |


In [123]:
--THIS IS A TRIANGLE--second vertex is reflection over x-axis (compared to first vertex)
F=(x^3-3*x-t)_R1;
x0=0;
t0=0;

s=10;
V=mutableIdentity(R1,2*s+1);
Coords=mutableIdentity(R1,2*s+1);
print V;
for j from 0 to 2*s do (
    for i from 0 to 2*s do (
        try(
            newSol=runAlgo(F,x0,t0,(-5+0.5*i)+(5-0.5*j)*ii,(-5+0.5*i)-(5-0.5*j)*ii);
        
            
        
            
        ) then (
            print(newSol.Coordinates_0);
            V_(i,j)= newSol.Coordinates_0;
        ) else (
            V_(i,j)= 9;
        );
        Coords_(i,j)=(-5+0.5*i)+(5-0.5*j)*ii;
    );
);
print V;
print Coords;



| 1 0 0 0 0 0 0 |
| 0 1 0 0 0 0 0 |
| 0 0 1 0 0 0 0 |
| 0 0 0 1 0 0 0 |
| 0 0 0 0 1 0 0 |
| 0 0 0 0 0 1 0 |
| 0 0 0 0 0 0 1 |


o123 = [KERNEL ENFORCED TIMEOUT]